In [3]:
clusters = [
            ['conspiracy','Android','atheism','Incels' ,'PurplePillDebate', 'IAmA',
 'canada', 'tifu', 'india', 'SubredditDrama', 'dataisbeautiful', 'pics',
 'LifeProTips', 'hiphopheads' ,'fantasyfootball', 'explainlikeimfive',
 'worldnews', 'SandersForPresident'],
    
            ['CanadaPolitics', 'spacex' ,'changemyview', 'NeutralPolitics',
 'personalfinance' ,'AskHistorians', 'history' ,'whatisthisthing', 'science',
 'Games' ,'philosophy', 'space' ,'Futurology' ,'syriancivilwar', 'legaladvice',
 'PoliticalDiscussion', 'AskTrumpSupporters' ,'TheSilphRoad' ,'Christianity',
 'DIY', 'OutOfTheLoop', 'UpliftingNews'],
    
            ['DestinyTheGame', 'hearthstone' ,'Overwatch' ,'jailbreak' ,'2007scape', 'wow'],
    
            ['CFB', 'me_irl', 'books', 'movies', 'nba' ,'nfl' ,'asoiaf', 'pokemon', 'MMA',
 'relationships' ,'AskWomen', 'food', 'pcmasterrace', 'Showerthoughts',
 'GlobalOffensiveTrade', 'pokemongo', 'leagueoflegends', 'depression',
 'gonewild', 'hillaryclinton', 'SuicideWatch', 'The_Donald', 'gaming',
 'GlobalOffensive' ,'anime', 'politics', 'photoshopbattles', 'television',
 'ShitRedditSays', 'GetMotivated' ,'aww', 'EnoughTrumpSpam', 'sex',
 'gameofthrones', 'TwoXChromosomes', 'funny', 'nottheonion', 'europe',
 'LateStageCapitalism', 'news', 'technology', 'soccerstreams', 'socialism'],
    
            ['churning', 'NSFW_GIF', 'pokemontrades', 'nosleep'],
    
            ['videos', 'OldSchoolCool', 'gifs'],
        
            ['AskReddit'], 
    
            ['BlackPeopleTwitter'],
            
            ['askscience'],
            
            ['creepyPMs']
        ]

In [39]:
import pandas as pd
comments = pd.read_csv('/Users/eshwar/Downloads/predictions_with_subreddit_english.csv')

study_cols = pd.read_csv('/Users/eshwar/Downloads/study_subreddits_english.csv', names = ['columns'])['columns']

study_subs = pd.read_csv('/Users/eshwar/Downloads/study_subreddits_english.csv', names = ['subreddit'])
def cleaner(string):
    return string.split('prediction_')[1]

study_subs['subreddit'] = study_subs['subreddit'].apply(cleaner)

In [42]:
### only retain predictions (columns) that come from the top 100 study subreddits and the actual comment
retain = ['comment', 'subreddit']
for col in study_cols[:100]:
    retain.append(col)

comments = comments[retain]

comments = comments[~(comments.comment == '__label__removed ')].drop_duplicates()

def comment_cleaner(string):
    return string.split('__label__removed ')[1]

comments['comment'] = comments['comment'].apply(comment_cleaner)

comments['total'] = comments[study_cols[:100]].sum(axis = 1)

comments = comments[comments.total > 0]

# Compile comments with high inter-cluster agreement

In [165]:
total = 0

c_num = 0

for c in clusters:
    print("Cluster #", c_num)
    total += len(c)
    print("Number of subreddits in cluster = ", len(c))
    
    cluster_cols = ['comment','subreddit']
    for c_sub in c:
        cluster_cols.append('prediction_' + c_sub)
    
    print("Number of cols = ", len(cluster_cols))

    col_ratio = 'c' + str(c_num) + '_ratio'

    ###1) calculate agreement to remove within cluster
    ###2) normalize by agreement to remove outside the cluster; smoothing by adding 0.1 incase out of cluster agreement = 0 (prevent divide by zero)
    comments[col_ratio] = (comments[cluster_cols].sum(axis = 1)/float(len(cluster_cols) - 2))/((0.1 + comments['total'] - comments[cluster_cols].sum(axis = 1))/(100.0 - len(cluster_cols) + 2.0))
    ###get comments agreement
    c_num += 1

##########        
print("Total #clusters = ", c_num, "; Total #subs = ", total)

Cluster # 0
Number of subreddits in cluster =  18
Number of cols =  20


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Cluster # 1
Number of subreddits in cluster =  22
Number of cols =  24
Cluster # 2
Number of subreddits in cluster =  6
Number of cols =  8
Cluster # 3
Number of subreddits in cluster =  43
Number of cols =  45
Cluster # 4
Number of subreddits in cluster =  4
Number of cols =  6
Cluster # 5
Number of subreddits in cluster =  3
Number of cols =  5
Cluster # 6
Number of subreddits in cluster =  1
Number of cols =  3
Cluster # 7
Number of subreddits in cluster =  1
Number of cols =  3
Cluster # 8
Number of subreddits in cluster =  1
Number of cols =  3
Cluster # 9
Number of subreddits in cluster =  1
Number of cols =  3
Total #clusters =  10 ; Total #subs =  100


In [172]:
comments.to_csv('final-cluster-agreements.csv', index = None)

In [171]:
# c_num = 5
# c = clusters[c_num - 1]
# cluster_cols = ['comment','subreddit']

# for c_sub in c:
#     cluster_cols.append('prediction_' + c_sub)
    
# temp = comments.sample(n=100)

# col_ratio = 'c' + str(c_num) + '_ratio'

# ###1) calculate agreement to remove within cluster
# ###2) normalize by agreement to remove outside the cluster

# temp[col_ratio] = (temp[cluster_cols].sum(axis = 1)/float(len(cluster_cols)))/((temp['total'] - temp[cluster_cols].sum(axis = 1))/(100.0 - len(cluster_cols)))

# temp.sort_values(by=[col_ratio], ascending=[0])[:10]

Index(['comment', 'subreddit', 'prediction_The_Donald', 'prediction_politics',
       'prediction_AskReddit', 'prediction_science', 'prediction_worldnews',
       'prediction_news', 'prediction_explainlikeimfive',
       'prediction_relationships',
       ...
       'c1_ratio', 'c2_ratio', 'c3_ratio', 'c4_ratio', 'c5_ratio', 'c6_ratio',
       'c7_ratio', 'c8_ratio', 'c9_ratio', 'c0_ratio'],
      dtype='object', length=113)

In [184]:
for col_num in range(10):
    col_ratio = 'c' + str(col_num) + '_ratio' 
    print(col_ratio)
    print(comments[col_ratio].max(), comments[col_ratio].min(), comments[col_ratio].mean())

c0_ratio
45.555555555555515 0.0 0.6735612346310941
c1_ratio
460.9090909090926 0.0 1.4171378250995297
c2_ratio
156.66666666666652 0.0 0.9116358370322889
c3_ratio
304.8837209302282 0.0 1.4895014851239166
c4_ratio
239.9999999999998 0.0 0.4295522642283078
c5_ratio
323.33333333333303 0.0 0.6808971972162586
c6_ratio
989.9999999999992 0.0 1.3224082304534697
c7_ratio
90.0 0.0 0.6896289002308035
c8_ratio
989.9999999999992 0.0 1.3244866057369713
c9_ratio
989.9999999999992 0.0 0.8561910289166184


In [196]:
query_cluster_num = 3
col_ratio = 'c' + str(query_cluster_num) + '_ratio'
print(col_ratio)
print(clusters[query_cluster_num])

c3_ratio
['CFB', 'me_irl', 'books', 'movies', 'nba', 'nfl', 'asoiaf', 'pokemon', 'MMA', 'relationships', 'AskWomen', 'food', 'pcmasterrace', 'Showerthoughts', 'GlobalOffensiveTrade', 'pokemongo', 'leagueoflegends', 'depression', 'gonewild', 'hillaryclinton', 'SuicideWatch', 'The_Donald', 'gaming', 'GlobalOffensive', 'anime', 'politics', 'photoshopbattles', 'television', 'ShitRedditSays', 'GetMotivated', 'aww', 'EnoughTrumpSpam', 'sex', 'gameofthrones', 'TwoXChromosomes', 'funny', 'nottheonion', 'europe', 'LateStageCapitalism', 'news', 'technology', 'soccerstreams', 'socialism']


In [284]:
###For the cluster under consideration, sample top 10K commments based on the 'col_ratio'
data = pd.DataFrame()
###consider top 1% of all comments based on ratio for the column
# top_comments = int(0.01 * comments.shape[0])
top_comments = 10000
# comments.sort_values([col_ratio], ascending = [0])[col_ratio][:top_comments]
data = comments.sort_values([col_ratio], ascending = [0])[['comment', 'subreddit', col_ratio]][:top_comments]

In [285]:
data.shape

(10000, 3)

# Topic modelling

In [206]:
data.head()

,comment,subreddit,c3_ratio
1333484,it was right in the dnc leaks today no it wasn...,politics,304.883721
1897377,sanders knows the gop avoided attacking him so...,politics,251.860465
1283741,it is less about the superdelegates and more a...,The_Donald,251.860465
1234902,in a sense yes but populist candidates like le...,CanadaPolitics,251.860465
457417,elo will pick the best available fit i e the a...,FIFA,251.860465


In [277]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cross_validation import train_test_split
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# vec = TfidfVectorizer(decode_error = 'ignore', token_pattern = "[a-zA-Z]{3,}",lowercase= True,stop_words = None, max_features =1000)
vec = TfidfVectorizer(decode_error = 'ignore', token_pattern = "[a-zA-Z]{3,}",lowercase= True,stop_words = stopwords.words('english'), max_features =1000)
# vec = CountVectorizer(token_pattern= "[a-zA-Z']{3,}",decode_error ='ignore', lowercase=True, stop_words = None,max_features = 1000, min_df=3)
# vec = CountVectorizer( token_pattern= "[a-zA-Z']{3,}",decode_error ='ignore', lowercase=True, stop_words = stopwords.words('english'),max_features = 1000)
sparse_matrice = vec.fit_transform(data.comment)
# X_train, X_test = train_test_split(sparse_matrice, test_size=0.20, random_state=42) ## 80 -20 split

lda = LatentDirichletAllocation(n_topics = 10, max_iter=100, topic_word_prior= 0.1, doc_topic_prior=1.0,learning_method='batch',n_jobs= 1,random_state=275)    
# topic_components = lda.fit_transform(X_train)
topic_components = lda.fit_transform(sparse_matrice)

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [288]:
list_topics = []

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        list_topics.append(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [289]:
print_top_words(lda,vec.get_feature_names(), 10) #print the top 10 words!

Topic #0:
states based probably take wrong try election possible war party
Topic #1:
would think much right want well far https com pretty
Topic #2:
state still though need something since used find left non
Topic #3:
actually good new less someone etc part better information either
Topic #4:
like get mean person two first yes saying means sense
Topic #5:
also use see without sure fact able best every case
Topic #6:
time way make work may question pay using money tax
Topic #7:
could many going thing said would system another specific likely
Topic #8:
one know say different example might years look difference never
Topic #9:
people even really point things lot problem however around end



In [280]:
test_topic_dist = pd.DataFrame(topic_components)
test_topic_dist['comment'] = list(data['comment'])
test_topic_dist['subreddit'] = list(data['subreddit'])

In [282]:
# lda.perplexity(X_test)
###get back the documents that score highly on each topic - show this to raters and ask them to label it!!!!

# predict topics for test data
# unnormalized comment-topic distribution

# # test_matrix = vec.transform(data.review)
# # test_topic_dist = pd.DataFrame(lda.transform(test_matrix))
# # test_topic_dist['comment'] = data.review

# test_matrix = vec.transform(data.comment)
# topic_dist_unnormalized = np.matrix(lda.transform(test_matrix))

# # normalize the distribution (only needed if you want to work with the probabilities)
# test_topic_dist = topic_dist_unnormalized/topic_dist_unnormalized.sum(axis = 1)

# test_topic_dist = pd.DataFrame(test_topic_dist)

# test_topic_dist['comment'] = data.comment

In [296]:
for query_topic in range(10):
    # test_topic_dist.sort_values(by = [query_topic], ascending=[False]).head()
    print("Topic #", query_topic, "; Top 10 topic words = ", list_topics[query_topic])
    count = 0

    sample_comments = test_topic_dist.drop_duplicates(subset='comment', keep="last").sort_values(by = [query_topic], ascending=[False])[:10]

    for index, row in sample_comments.iterrows():
        comment = row['comment']
        subreddit = row['subreddit']
    # for comment in test_topic_dist.sort_values(by = [query_topic], ascending=[False])[:10]['comment'].drop_duplicates():
    # for comment in test_topic_dist.sort_values(by = [query_topic], ascending=[False])[:100]['comment'].drop_duplicates().sample(n = 10):
        print("Example: ", count, " - from r/", subreddit)
        print(comment)
        print("\n")
        count+=1

Topic # 0 ; Top 10 topic words =  states based probably take wrong try election possible war party
Example:  0  - from r/ AskHistorians
i couldn t attest to the main point of your question but it is interesting to note that the emancipation proclamation signed 1 jan 1863 only applied to slaves in the south listed as rebellious states in the text where union slaves were freed upon ratification of the 13th amendment nearly 3 years later in dec of 1865 it also did not grant citizenship to freed slaves but encouraged their service in the war the emancipation was declared as an objective of war to make sure the intent is well known a few months prior to the proclamation warning was given to the rebellious states and they were not impressed https www archives gov exhibits featured_documents emancipation_proclamation


Example:  1  - from r/ linguistics
correct me if i m wrong but doesn t it have to do with the way canada and the united states were settled and their shared history the northea

# Generate data for coding!!!

In [336]:
cluster_level_topics = []
cluster_level_example_comments = []

for query_cluster_num in range(10):
    col_ratio = 'c' + str(query_cluster_num) + '_ratio'
    print("Working on Cluster #", query_cluster_num)
    print("Subreddits in this cluster are: ", clusters[query_cluster_num])

    ###For the cluster under consideration, sample top 10K commments based on the 'col_ratio'
    data = pd.DataFrame()
    ###consider top 1% of all comments based on ratio for the column
    top_comments = int(0.01 * comments.shape[0])
#     top_comments = 1000
    # comments.sort_values([col_ratio], ascending = [0])[col_ratio][:top_comments]
    data = comments.sort_values([col_ratio], ascending = [0])[['comment', 'subreddit', col_ratio]][:top_comments]

    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.decomposition import LatentDirichletAllocation
    from sklearn.cross_validation import train_test_split
    import numpy as np
    from nltk.corpus import stopwords
    from sklearn.feature_extraction.text import TfidfVectorizer

    # vec = TfidfVectorizer(decode_error = 'ignore', token_pattern = "[a-zA-Z]{3,}",lowercase= True,stop_words = None, max_features =1000)
    vec = TfidfVectorizer(decode_error = 'ignore', token_pattern = "[a-zA-Z]{3,}",lowercase= True,stop_words = stopwords.words('english'), max_features =1000)
    # vec = CountVectorizer(token_pattern= "[a-zA-Z']{3,}",decode_error ='ignore', lowercase=True, stop_words = None,max_features = 1000, min_df=3)
    # vec = CountVectorizer( token_pattern= "[a-zA-Z']{3,}",decode_error ='ignore', lowercase=True, stop_words = stopwords.words('english'),max_features = 1000)
    sparse_matrice = vec.fit_transform(data.comment)
    # X_train, X_test = train_test_split(sparse_matrice, test_size=0.20, random_state=42) ## 80 -20 split

    lda = LatentDirichletAllocation(n_topics = 10, max_iter=100, topic_word_prior= 0.1, doc_topic_prior=1.0,learning_method='batch',n_jobs= 1,random_state=275)    
    # topic_components = lda.fit_transform(X_train)
    topic_components = lda.fit_transform(sparse_matrice)

    list_topics = []

    def print_top_words(model, feature_names, n_top_words):
        for topic_idx, topic in enumerate(model.components_):
    #         print("Topic #%d:" % topic_idx)
    #         print(" ".join([feature_names[i]
    #                         for i in topic.argsort()[:-n_top_words - 1:-1]]))
            list_topics.append(" ".join([feature_names[i]
                            for i in topic.argsort()[:-n_top_words - 1:-1]]))
    #     print()

    print_top_words(lda,vec.get_feature_names(), 10) #print the top 10 words!

    cluster_level_topics.append(list_topics)
    
    test_topic_dist = pd.DataFrame(topic_components)
    test_topic_dist['comment'] = list(data['comment'])
    test_topic_dist['subreddit'] = list(data['subreddit'])

    topic_level_example_comments = []
    for query_topic in range(10):
        # test_topic_dist.sort_values(by = [query_topic], ascending=[False]).head()
        print("Topic #", query_topic, "; Top 10 topic words = ", list_topics[query_topic])
        count = 0

        sample_comments = test_topic_dist.drop_duplicates(subset='comment', keep="last").sort_values(by = [query_topic], ascending=[False])[:10]
        topic_level_example_comments.append(list(sample_comments['comment']))
        for index, row in sample_comments.iterrows():
            comment = row['comment']
            subreddit = row['subreddit']
        # for comment in test_topic_dist.sort_values(by = [query_topic], ascending=[False])[:10]['comment'].drop_duplicates():
        # for comment in test_topic_dist.sort_values(by = [query_topic], ascending=[False])[:100]['comment'].drop_duplicates().sample(n = 10):
            print("Example: ", count, " - from r/", subreddit)
            print(comment)
            print("\n")
            count+=1
            
    cluster_level_example_comments.append(topic_level_example_comments)

Working on Cluster # 0
Subreddits in this cluster are:  ['conspiracy', 'Android', 'atheism', 'Incels', 'PurplePillDebate', 'IAmA', 'canada', 'tifu', 'india', 'SubredditDrama', 'dataisbeautiful', 'pics', 'LifeProTips', 'hiphopheads', 'fantasyfootball', 'explainlikeimfive', 'worldnews', 'SandersForPresident']


/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic # 0 ; Top 10 topic words =  found look work record want thought correcting ctr use man
Example:  0  - from r/ politics
hey man thanks for correcting the record


Example:  1  - from r/ politics
he s just correcting the record friend


Example:  2  - from r/ LateStageCapitalism
you do realize that you need to work in any economic system currently in place right


Example:  3  - from r/ news
thanks for correcting the record on that bro


Example:  4  - from r/ politics
thanks for correcting the record for me bro


Example:  5  - from r/ politics
thanks for correcting the record with that statistic bro


Example:  6  - from r/ worldnews
thanks for correcting the record buddy


Example:  7  - from r/ politics
thanks for correcting the record friend


Example:  8  - from r/ worldnews
hey thanks for correcting the record


Example:  9  - from r/ politics
thank you for correcting the record friend


Topic # 1 ; Top 10 topic words =  thank much god wow yes keep damn wait first always
Exa

Example:  0  - from r/ uwaterloo
spread your semen everywhere all over the bathroom and hope a little bit goes inside of her


Example:  1  - from r/ science
hoky fucj the whole threads gone


Example:  2  - from r/ ShitRedditSays
does it though i think that s pretty heavily debated some biased translations there s been


Example:  3  - from r/ blunderyears
you looked great at least


Example:  4  - from r/ space
great soon all of our scientists will be goes r worshippers


Example:  5  - from r/ nfl
definitely a hack we will see where this goes now


Example:  6  - from r/ ShitRedditSays
don t worry america will be great again soon


Example:  7  - from r/ wow
hope to see you on r gonewild soon


Example:  8  - from r/ SuicideWatch
i m so sorry see you soon


Example:  9  - from r/ science
so instead of seeing a deer eating your garden you d rather see a cougar eating your pet


Topic # 7 ; Top 10 topic words =  need take right also space problem let hard safe getting
Example:  0  - f

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


...

Example:  0  - from r/ worldnews
i m on the fence the eu s domination of european trade compels countries to remain members against their own inclinations i don t think that the uk should be rewarded for withdrawing but i think that nations should have an option to leave without destroying themselves


Example:  1  - from r/ Games
i think technically you d need jets for that rather than cars


Example:  2  - from r/ AskReddit
need to learn programming for my career but i don t think i can


Example:  3  - from r/ AskReddit
weirdest things just don t get in there unless you intend them to enter which is one of the reasons not to circumcise a boy his foreskin protects against contamination of the urethra


Example:  4  - from r/ linux
it s not a bug it s a feature it does exactly what it says it does this is just bad design working as intended in my experience complaining to freedesktop that they prioritize ohh windows cannot get confused over actual design sanity and security does squat

Example:  0  - from r/ science
i remember having this experiment in my middle school health class they attached a bracelet and key to my wrist gave me the baby and baby seat i was to stick the key in its back when it started crying and they would record how often i neglected it as well as how long so i carried that thing to my remaining classes that day and onto the school bus with me i remember being so embarrassed as i was whacking kids in the head with the baby seat through the isle when i got home the thing would not shut up it was crying every few minutes i stuck the key in its back it stopped but not for long eventually i fell asleep because it put it in another room under some sheets i slept through its crying all night and apparently it woke my mom up who brought it to my room and used the key on my wrist to shut it up i don t remember this though just what she told me i got an a


Example:  1  - from r/ AskReddit
i m not a teacher but i was the one unintentionally raising a to

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic # 0 ; Top 10 topic words =  know well actually anything far question looks works everything exactly
Example:  0  - from r/ science
as far as i know the color blue absorbs the most sunlight that s why solar panels are blue as well


Example:  1  - from r/ whatisthisthing
well that answers a question i didn t know i had


Example:  2  - from r/ GlobalOffensiveTrade
i ve heard about it but don t know how it works


Example:  3  - from r/ hillaryclinton
but yard signs but rallies don t they know that s what counts not standing in line to vote


Example:  4  - from r/ gameofthrones
rickon is the replacement for the sansa double in the book but the show is so far off the rails than anything is possible now


Example:  5  - from r/ syriancivilwar
leith s dad is far more pro assad than leith is by which i mean leith actually recognizes reality for the most part


Example:  6  - from r/ boardgames
catchup mechanism triggered power grid is my choice as well though


Example:  7  - from r/ 

Example:  0  - from r/ BravoRealHousewives
i hate to admit it but i love the long bobs that both kelly and icky vicki have


Example:  1  - from r/ science
but how do you go to sleep in the evening even if i nap for half a hour i ll never go back to sleep until at least 1 2 am


Example:  2  - from r/ communism101
communism or more accurately socialism which is the first step to communism doesn t have a centralized state or at the very least not a large one many socialists communists are against having a large state afterall it s not socialism if the government owns the means of production


Example:  3  - from r/ movies
seriously at least see if once or twice in theatres before you torrent the movies aren t that expensive


Example:  4  - from r/ explainlikeimfive
i m not patting myself on the back or anything but i m at least too big to fit in a urethra lol


Example:  5  - from r/ fantasyfootball
take jennings for the long term unless your rbs are stacked and healthy he s undervalue

In [337]:
pd.DataFrame(cluster_level_topics).to_csv('cluster_level_topics.csv')
pd.DataFrame(cluster_level_example_comments).to_csv('cluster_level_example_comments.csv')
print("DONE!")

DONE!


In [ ]:
###started at 9:05PM, April 3

# MACRO NORMS

In [368]:
macro = comments[['comment', 'subreddit', 'total']]
macro = macro.drop_duplicates(subset='comment', keep="last")
macro_limit = int(0.01 * macro.shape[0])  ###top 1 percentile - gives agreement > 85%
macro_comments = macro.sort_values(by = ['total'], ascending=[False])[:macro_limit]

In [369]:
macro['total'].max(),macro['total'].min(),macro['total'].mean()

(100, 1, 52.39275657773282)

In [373]:
macro_topics = []
macro_example_comments = []

print("Working on Macro cluster")
print("Subreddits in this cluster are: 100" )

###For the cluster under consideration, sample top 10K commments based on the 'col_ratio'
data = pd.DataFrame()
###consider top 1% of all comments based on ratio for the column
data = macro_comments

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cross_validation import train_test_split
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# vec = TfidfVectorizer(decode_error = 'ignore', token_pattern = "[a-zA-Z]{3,}",lowercase= True,stop_words = None, max_features =1000)
vec = TfidfVectorizer(decode_error = 'ignore', token_pattern = "[a-zA-Z]{3,}",lowercase= True,stop_words = stopwords.words('english'), max_features =1000)
# vec = CountVectorizer(token_pattern= "[a-zA-Z']{3,}",decode_error ='ignore', lowercase=True, stop_words = None,max_features = 1000, min_df=3)
# vec = CountVectorizer( token_pattern= "[a-zA-Z']{3,}",decode_error ='ignore', lowercase=True, stop_words = stopwords.words('english'),max_features = 1000)
sparse_matrice = vec.fit_transform(data.comment)
# X_train, X_test = train_test_split(sparse_matrice, test_size=0.20, random_state=42) ## 80 -20 split
print("Done Vectorizing...")
lda = LatentDirichletAllocation(n_topics = 10, max_iter=100, topic_word_prior= 0.1, doc_topic_prior=1.0,learning_method='batch',n_jobs= 1,random_state=275)    
# topic_components = lda.fit_transform(X_train)
topic_components = lda.fit_transform(sparse_matrice)

print("Done fitting LDA...")
list_topics = []

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
#         print("Topic #%d:" % topic_idx)
#         print(" ".join([feature_names[i]
#                         for i in topic.argsort()[:-n_top_words - 1:-1]]))
        list_topics.append(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
#     print()

print_top_words(lda,vec.get_feature_names(), 10) #print the top 10 words!

macro_topics.append(list_topics)

test_topic_dist = pd.DataFrame(topic_components)
test_topic_dist['comment'] = list(data['comment'])
test_topic_dist['subreddit'] = list(data['subreddit'])

topic_level_example_comments = []
for query_topic in range(10):
    # test_topic_dist.sort_values(by = [query_topic], ascending=[False]).head()
    print("Topic #", query_topic, "; Top 10 topic words = ", list_topics[query_topic])
    count = 0

    sample_comments = test_topic_dist.drop_duplicates(subset='comment', keep="last").sort_values(by = [query_topic], ascending=[False])[:10]
    topic_level_example_comments.append(list(sample_comments['comment']))
    for index, row in sample_comments.iterrows():
        comment = row['comment']
        subreddit = row['subreddit']
    # for comment in test_topic_dist.sort_values(by = [query_topic], ascending=[False])[:10]['comment'].drop_duplicates():
    # for comment in test_topic_dist.sort_values(by = [query_topic], ascending=[False])[:100]['comment'].drop_duplicates().sample(n = 10):
        print("Example: ", count, " - from r/", subreddit)
        print(comment)
        print("\n")
        count+=1

macro_example_comments.append(topic_level_example_comments)

Working on Macro cluster
Subreddits in this cluster are: 100
Done Vectorizing...


/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Done fitting LDA...
Topic # 0 ; Top 10 topic words =  cunt idiot lol dumb pussy one cares die nobody wow
Example:  0  - from r/ The_Donald
what a dumb cunt lol what a pussy


Example:  1  - from r/ relationships
nobody cares you dumb cunt


Example:  2  - from r/ pics
wow kinda dumb u and your cunt daughter


Example:  3  - from r/ The_Donald
no one gives a fuck what this batshit community thinks


Example:  4  - from r/ The_Donald
savage this pussy is a bumbling idiot lol


Example:  5  - from r/ offmychest
nobody cares you pussy faget


Example:  6  - from r/ IAmA
you re a dumb cunt lol


Example:  7  - from r/ worldnews
you are one dumb cunt


Example:  8  - from r/ deadbydaylight
u leave nobody wants u here cuckface


Example:  9  - from r/ AskReddit
no one cares cunt


Topic # 1 ; Top 10 topic words =  reddit bot totesmessenger please https comments thread com someone another
Example:  0  - from r/ BustyPetite
i m a bot bleep bloop someone has linked to this thread from another pl

Example:  0  - from r/ politics
holy shit this sub is filled with pathetic cancer go hang yourselves


Example:  1  - from r/ TwoXChromosomes
why does this piece of shit sub reddit show up on my front page


Example:  2  - from r/ pcmasterrace
you re so cool you hang dead animal skulls up you re a cunt


Example:  3  - from r/ hearthstone
your english sucks ass kys


Example:  4  - from r/ SandersForPresident
bye bye bernie you lying piece of shit 200mil to hillary the revolution is dead


Example:  5  - from r/ SubredditDrama
drain the swamp u spez is a kek kekadooddledoo fuck reddit this site sucks


Example:  6  - from r/ GetMotivated
seriously kill yourself do everyone a favor


Example:  7  - from r/ EnoughTrumpSpam
what the fuck makes you think this site deserves it look what sub you are in for what the post is you fucking silly faggot kill yourself


Example:  8  - from r/ psychology
fuck you reddit you sanders loving shit fuckers


Example:  9  - from r/ skyrimmods
you re a fag

In [374]:
pd.DataFrame(cluster_level_topics).to_csv('macro_topics.csv')
pd.DataFrame(cluster_level_example_comments).to_csv('macro_example_comments.csv')
print("DONE!")

DONE!


In [371]:
macro_comments.shape, macro.shape

((25296, 3), (2529633, 3))

In [375]:
macro_topics = []
macro_example_comments = []

print("Working on Macro cluster")
print("Subreddits in this cluster are: 100" )

###For the cluster under consideration, sample top 10K commments based on the 'col_ratio'
data = pd.DataFrame()
###consider top 1% of all comments based on ratio for the column
data = macro_comments

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cross_validation import train_test_split
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# vec = TfidfVectorizer(decode_error = 'ignore', token_pattern = "[a-zA-Z]{3,}",lowercase= True,stop_words = None, max_features =1000)
vec = TfidfVectorizer(decode_error = 'ignore', token_pattern = "[a-zA-Z]{3,}",lowercase= True,stop_words = stopwords.words('english'), max_features =1000)
# vec = CountVectorizer(token_pattern= "[a-zA-Z']{3,}",decode_error ='ignore', lowercase=True, stop_words = None,max_features = 1000, min_df=3)
# vec = CountVectorizer( token_pattern= "[a-zA-Z']{3,}",decode_error ='ignore', lowercase=True, stop_words = stopwords.words('english'),max_features = 1000)
sparse_matrice = vec.fit_transform(data.comment)
# X_train, X_test = train_test_split(sparse_matrice, test_size=0.20, random_state=42) ## 80 -20 split
print("Done Vectorizing...")
lda = LatentDirichletAllocation(n_topics = 15, max_iter=100, topic_word_prior= 0.1, doc_topic_prior=1.0,learning_method='batch',n_jobs= 1,random_state=275)    
# topic_components = lda.fit_transform(X_train)
topic_components = lda.fit_transform(sparse_matrice)

print("Done fitting LDA...")
list_topics = []

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
#         print("Topic #%d:" % topic_idx)
#         print(" ".join([feature_names[i]
#                         for i in topic.argsort()[:-n_top_words - 1:-1]]))
        list_topics.append(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
#     print()

print_top_words(lda,vec.get_feature_names(), 10) #print the top 10 words!

macro_topics.append(list_topics)

test_topic_dist = pd.DataFrame(topic_components)
test_topic_dist['comment'] = list(data['comment'])
test_topic_dist['subreddit'] = list(data['subreddit'])

topic_level_example_comments = []
for query_topic in range(15):
    # test_topic_dist.sort_values(by = [query_topic], ascending=[False]).head()
    print("Topic #", query_topic, "; Top 10 topic words = ", list_topics[query_topic])
    count = 0

    sample_comments = test_topic_dist.drop_duplicates(subset='comment', keep="last").sort_values(by = [query_topic], ascending=[False])[:10]
    topic_level_example_comments.append(list(sample_comments['comment']))
    for index, row in sample_comments.iterrows():
        comment = row['comment']
        subreddit = row['subreddit']
    # for comment in test_topic_dist.sort_values(by = [query_topic], ascending=[False])[:10]['comment'].drop_duplicates():
    # for comment in test_topic_dist.sort_values(by = [query_topic], ascending=[False])[:100]['comment'].drop_duplicates().sample(n = 10):
        print("Example: ", count, " - from r/", subreddit)
        print(comment)
        print("\n")
        count+=1

macro_example_comments.append(topic_level_example_comments)

Working on Macro cluster
Subreddits in this cluster are: 100
Done Vectorizing...


/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Done fitting LDA...
Topic # 0 ; Top 10 topic words =  cunt idiot pussy wow troll read dumbass grow game calling
Example:  0  - from r/ books
wow you re an idiot grow up


Example:  1  - from r/ Fitness
fuck you you lazy cunt read it yourself


Example:  2  - from r/ leagueoflegends
you are what s wrong with this community you re a bitch and a pussy grow fucking balls and stop crying over everything


Example:  3  - from r/ worldnews
you re an idiot read the fucking program before spouting shit dumbass


Example:  4  - from r/ pics
wow kinda dumb u and your cunt daughter


Example:  5  - from r/ pokemongo
pussy idiot


Example:  6  - from r/ news
wow you re a pussy


Example:  7  - from r/ Calgary
you re an idiot wow


Example:  8  - from r/ AskReddit
wow aren t you a cunt


Example:  9  - from r/ worldnews
wow you are a cunt


Topic # 1 ; Top 10 topic words =  fucking retard sub pathetic ban scum hey mod banned fake
Example:  0  - from r/ The_Donald
remove u mannoslimmins he s unfit to

Example:  0  - from r/ investing
go kill yourself you useless sack of meat


Example:  1  - from r/ gtaonline
stop whining fucking cry baby


Example:  2  - from r/ conspiracy
get off your fucking high horse princess daily mail is shit you shill


Example:  3  - from r/ hearthstone
why are you whining stfu you whining sack of shit


Example:  4  - from r/ ffxiv
fucking cry about it you fucking baby


Example:  5  - from r/ sanfrancisco
you fucking shill stfu


Example:  6  - from r/ The_Donald
bullshit you fucking shill


Example:  7  - from r/ videos
nice repost you fucking queer


Example:  8  - from r/ The_Donald
fucking garbage kill yourself


Example:  9  - from r/ The_Donald
hahahahaha fucking kill yourselves sjw libtards


Topic # 10 ; Top 10 topic words =  trump get post donald fucked want bunch supporters guy delete
Example:  0  - from r/ The_Donald
donald trump will lose this election if you want to prove that you know for a fact that trump will lose this election then please

no one cares you child


Example:  2  - from r/ PhantomForces
nobody cares about your stupid stats


Example:  3  - from r/ news
yeah and i will next time and many people did because of stupid fucking liberals like you who call everyone you disagree with racist


Example:  4  - from r/ The_Donald
just kidding youre a fucking moron


Example:  5  - from r/ pcmasterrace
nobody cares what you think fuck off


Example:  6  - from r/ GlobalOffensive
lol funny copypasta or you re a fucking moron either way i laugh


Example:  7  - from r/ nba
no one cares what you think kill yourself


Example:  8  - from r/ playrust
no one cares you stupid gook cunt


Example:  9  - from r/ hillaryclinton
one dumb comment who cares




In [377]:
1

1

In [379]:
macro_topics = []
macro_example_comments = []

print("Working on Macro cluster")
print("Subreddits in this cluster are: 100" )

###For the cluster under consideration, sample top 10K commments based on the 'col_ratio'
data = pd.DataFrame()
###consider top 1% of all comments based on ratio for the column
data = macro_comments

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cross_validation import train_test_split
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# vec = TfidfVectorizer(decode_error = 'ignore', token_pattern = "[a-zA-Z]{3,}",lowercase= True,stop_words = None, max_features =1000)
vec = TfidfVectorizer(decode_error = 'ignore', token_pattern = "[a-zA-Z]{3,}",lowercase= True,stop_words = stopwords.words('english'), max_features =1000)
# vec = CountVectorizer(token_pattern= "[a-zA-Z']{3,}",decode_error ='ignore', lowercase=True, stop_words = None,max_features = 1000, min_df=3)
# vec = CountVectorizer( token_pattern= "[a-zA-Z']{3,}",decode_error ='ignore', lowercase=True, stop_words = stopwords.words('english'),max_features = 1000)
sparse_matrice = vec.fit_transform(data.comment)
# X_train, X_test = train_test_split(sparse_matrice, test_size=0.20, random_state=42) ## 80 -20 split
print("Done Vectorizing...")
lda = LatentDirichletAllocation(n_topics = 20, max_iter=100, topic_word_prior= 0.1, doc_topic_prior=1.0,learning_method='batch',n_jobs= 1,random_state=275)    
# topic_components = lda.fit_transform(X_train)
topic_components = lda.fit_transform(sparse_matrice)

print("Done fitting LDA...")
list_topics = []

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
#         print("Topic #%d:" % topic_idx)
#         print(" ".join([feature_names[i]
#                         for i in topic.argsort()[:-n_top_words - 1:-1]]))
        list_topics.append(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
#     print()

print_top_words(lda,vec.get_feature_names(), 10) #print the top 10 words!

macro_topics.append(list_topics)

test_topic_dist = pd.DataFrame(topic_components)
test_topic_dist['comment'] = list(data['comment'])
test_topic_dist['subreddit'] = list(data['subreddit'])

topic_level_example_comments = []
for query_topic in range(20):
    # test_topic_dist.sort_values(by = [query_topic], ascending=[False]).head()
    print("Topic #", query_topic, "; Top 10 topic words = ", list_topics[query_topic])
    count = 0

    sample_comments = test_topic_dist.drop_duplicates(subset='comment', keep="last").sort_values(by = [query_topic], ascending=[False])[:10]
    topic_level_example_comments.append(list(sample_comments['comment']))
    for index, row in sample_comments.iterrows():
        comment = row['comment']
        subreddit = row['subreddit']
    # for comment in test_topic_dist.sort_values(by = [query_topic], ascending=[False])[:10]['comment'].drop_duplicates():
    # for comment in test_topic_dist.sort_values(by = [query_topic], ascending=[False])[:100]['comment'].drop_duplicates().sample(n = 10):
        print("Example: ", count, " - from r/", subreddit)
        print(comment)
        print("\n")
        count+=1

macro_example_comments.append(topic_level_example_comments)

Working on Macro cluster
Subreddits in this cluster are: 100
Done Vectorizing...


/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Done fitting LDA...
Topic # 0 ; Top 10 topic words =  idiot pussy fucking wow troll read dumbass grow anything calling
Example:  0  - from r/ books
wow you re an idiot grow up


Example:  1  - from r/ worldnews
you re an idiot read the fucking program before spouting shit dumbass


Example:  2  - from r/ teenagers
wow arent you a fucking pussy


Example:  3  - from r/ leagueoflegends
grow a pair u fucking pussy


Example:  4  - from r/ worldnews
you re a fucking idiot wow


Example:  5  - from r/ HistoryPorn
you re a fucking idiot read marx


Example:  6  - from r/ news
read his tweets from this morning you fucking idiot


Example:  7  - from r/ politics
you learning how to read is not grammar you fucking idiot


Example:  8  - from r/ news
you left cos you re a fucking pussy


Example:  9  - from r/ pokemongo
pussy idiot


Topic # 1 ; Top 10 topic words =  fuck retard pathetic ban hey fake please spez getting jews
Example:  0  - from r/ WTF
hey r bakedndelicious go fuck yourself d mmm

Example:  0  - from r/ pics
eat a dick you reposting fucktard


Example:  1  - from r/ politics
fuck turnip and anyone who supports him you people are traitors to your country


Example:  2  - from r/ legaladvice
eat a dick u scummy wannabe lawyer our system sucks dick u fucking moron it should be avoided at all costs


Example:  3  - from r/ ApocalypseRising
fuck you and ar2 will suck dick you fucking trash shit


Example:  4  - from r/ worldnews
why don t you suck your mum s dick


Example:  5  - from r/ SandersForPresident
ok go suck his dick


Example:  6  - from r/ videos
suck a dick homo


Example:  7  - from r/ lgbt
can i suck your dick


Example:  8  - from r/ CoDCompetitive
go suck his dick


Example:  9  - from r/ MMA
bruh u suck dick


Topic # 12 ; Top 10 topic words =  reddit bot totesmessenger thread please someone another vote rules place
Example:  0  - from r/ BustyPetite
i m a bot bleep bloop someone has linked to this thread from another place on reddit r bestofnsfw r 

In [381]:
for query_topic in range(20):
    # test_topic_dist.sort_values(by = [query_topic], ascending=[False]).head()
    print("Topic #", query_topic, "; Top 10 topic words = ", list_topics[query_topic])
    count = 0

    sample_comments = test_topic_dist.drop_duplicates(subset='comment', keep="last").sort_values(by = [query_topic], ascending=[False])[:10]
    topic_level_example_comments.append(list(sample_comments['comment']))
    for index, row in sample_comments.iterrows():
        comment = row['comment']
        subreddit = row['subreddit']
    # for comment in test_topic_dist.sort_values(by = [query_topic], ascending=[False])[:10]['comment'].drop_duplicates():
    # for comment in test_topic_dist.sort_values(by = [query_topic], ascending=[False])[:100]['comment'].drop_duplicates().sample(n = 10):
        print("Example: ", count, " - from r/", subreddit)
        print(comment)
        print("\n")
        count+=1

Topic # 0 ; Top 10 topic words =  idiot pussy fucking wow troll read dumbass grow anything calling
Example:  0  - from r/ books
wow you re an idiot grow up


Example:  1  - from r/ worldnews
you re an idiot read the fucking program before spouting shit dumbass


Example:  2  - from r/ teenagers
wow arent you a fucking pussy


Example:  3  - from r/ leagueoflegends
grow a pair u fucking pussy


Example:  4  - from r/ worldnews
you re a fucking idiot wow


Example:  5  - from r/ HistoryPorn
you re a fucking idiot read marx


Example:  6  - from r/ news
read his tweets from this morning you fucking idiot


Example:  7  - from r/ politics
you learning how to read is not grammar you fucking idiot


Example:  8  - from r/ news
you left cos you re a fucking pussy


Example:  9  - from r/ pokemongo
pussy idiot


Topic # 1 ; Top 10 topic words =  fuck retard pathetic ban hey fake please spez getting jews
Example:  0  - from r/ WTF
hey r bakedndelicious go fuck yourself d mmmmmkay great l


Exam

Example:  0  - from r/ news
mlk was an uncle tom and you re a fucking white male thus wrong


Example:  1  - from r/ Incels
is this real what the fuck are people actually this fucking retarded


Example:  2  - from r/ funny
fuck the president fuck their safety fuck the guy who took the picture fuck meowth fuck pokemon and if you support israel and zionism fuck you


Example:  3  - from r/ tifu
retarded mod


Example:  4  - from r/ wow
because you are actually retarded


Example:  5  - from r/ oculus
find out yourself you fucking twat


Example:  6  - from r/ BlackPeopleTwitter
you re a white male teen


Example:  7  - from r/ UNBGBBIIVCHIDCTIICBG
girls are retarded


Example:  8  - from r/ NoMansSkyTheGame
thats retarded


Example:  9  - from r/ Overwatch
is this guy fucking retarded


Topic # 16 ; Top 10 topic words =  trump racist women scum hate supporters see really karma cucks
Example:  0  - from r/ BlackPeopleTwitter
tfw you pretend all trump supporters are racist


Example:  1  

In [382]:
cluster_level_topics = []
cluster_level_example_comments = []

for query_cluster_num in [8]:
    col_ratio = 'c' + str(query_cluster_num) + '_ratio'
    print("Working on Cluster #", query_cluster_num)
    print("Subreddits in this cluster are: ", clusters[query_cluster_num])

    ###For the cluster under consideration, sample top 10K commments based on the 'col_ratio'
    data = pd.DataFrame()
    ###consider top 1% of all comments based on ratio for the column
    top_comments = int(0.01 * comments.shape[0])
#     top_comments = 1000
    # comments.sort_values([col_ratio], ascending = [0])[col_ratio][:top_comments]
    data = comments.sort_values([col_ratio], ascending = [0])[['comment', 'subreddit', col_ratio]][:top_comments]

    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.decomposition import LatentDirichletAllocation
    from sklearn.cross_validation import train_test_split
    import numpy as np
    from nltk.corpus import stopwords
    from sklearn.feature_extraction.text import TfidfVectorizer

    # vec = TfidfVectorizer(decode_error = 'ignore', token_pattern = "[a-zA-Z]{3,}",lowercase= True,stop_words = None, max_features =1000)
    vec = TfidfVectorizer(decode_error = 'ignore', token_pattern = "[a-zA-Z]{3,}",lowercase= True,stop_words = stopwords.words('english'), max_features =1000)
    # vec = CountVectorizer(token_pattern= "[a-zA-Z']{3,}",decode_error ='ignore', lowercase=True, stop_words = None,max_features = 1000, min_df=3)
    # vec = CountVectorizer( token_pattern= "[a-zA-Z']{3,}",decode_error ='ignore', lowercase=True, stop_words = stopwords.words('english'),max_features = 1000)
    sparse_matrice = vec.fit_transform(data.comment)
    # X_train, X_test = train_test_split(sparse_matrice, test_size=0.20, random_state=42) ## 80 -20 split

    lda = LatentDirichletAllocation(n_topics = 10, max_iter=100, topic_word_prior= 0.1, doc_topic_prior=1.0,learning_method='batch',n_jobs= 1,random_state=275)    
    # topic_components = lda.fit_transform(X_train)
    topic_components = lda.fit_transform(sparse_matrice)

    list_topics = []

    def print_top_words(model, feature_names, n_top_words):
        for topic_idx, topic in enumerate(model.components_):
    #         print("Topic #%d:" % topic_idx)
    #         print(" ".join([feature_names[i]
    #                         for i in topic.argsort()[:-n_top_words - 1:-1]]))
            list_topics.append(" ".join([feature_names[i]
                            for i in topic.argsort()[:-n_top_words - 1:-1]]))
    #     print()

    print_top_words(lda,vec.get_feature_names(), 10) #print the top 10 words!

    cluster_level_topics.append(list_topics)
    
    test_topic_dist = pd.DataFrame(topic_components)
    test_topic_dist['comment'] = list(data['comment'])
    test_topic_dist['subreddit'] = list(data['subreddit'])

    topic_level_example_comments = []
    for query_topic in range(10):
        # test_topic_dist.sort_values(by = [query_topic], ascending=[False]).head()
        print("Topic #", query_topic, "; Top 10 topic words = ", list_topics[query_topic])
        count = 0

        sample_comments = test_topic_dist.drop_duplicates(subset='comment', keep="last").sort_values(by = [query_topic], ascending=[False])[:10]
        topic_level_example_comments.append(list(sample_comments['comment']))
        for index, row in sample_comments.iterrows():
            comment = row['comment']
            subreddit = row['subreddit']
        # for comment in test_topic_dist.sort_values(by = [query_topic], ascending=[False])[:10]['comment'].drop_duplicates():
        # for comment in test_topic_dist.sort_values(by = [query_topic], ascending=[False])[:100]['comment'].drop_duplicates().sample(n = 10):
            print("Example: ", count, " - from r/", subreddit)
            print(comment)
            print("\n")
            count+=1
            
    cluster_level_example_comments.append(topic_level_example_comments)

Working on Cluster # 8
Subreddits in this cluster are:  ['askscience']


/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic # 0 ; Top 10 topic words =  people lot say thing way different things yeah many use
Example:  0  - from r/ linguistics
ianal but personally i have found that reading writing listening speaking are four different skills each must be practiced to improve and you can have varying levels in each in my particular case i get more chances to speak than to listen and so it is actually easier for me to speak in my second language than it is to understand natives speaking


Example:  1  - from r/ videos
yeah i m gonna say that s different from robbery


Example:  2  - from r/ TwoXChromosomes
yep wrote that same thing just with a lot more words because i forgot that term


Example:  3  - from r/ The_Donald
as a leftist i ll concede that he certainly stands against a lot of the things i stand for


Example:  4  - from r/ science
considering that chinese has a very simple grammatical structure built up by context rather than grammatical cases e g past future tense and it s all individual symb

Example:  0  - from r/ AskSocialScience
i ve studied psychology for five years and he has not been mentioned anywhere that i can recall


Example:  1  - from r/ gaming
for the last 3 years i keep saying to myself i m not buying cod but i always end buying it but this year i m def not buying it


Example:  2  - from r/ askscience
lysol in particular kills the root cause of odors by killing the germs that generate them in my personal experience lysol works very well in taking care of athlete s foot it absolutely stings like hell but is instant death to the fungi again in my personal experience the usual athlete s foot sprays work too the difference being they take longer to get the job done but don t hurt nearly as much on first application


Example:  3  - from r/ science
vaping causes inflammation and exacerbates asthma symptoms first i ve heard of this i quit smoking and began vaping 6 months ago away from public of course to avoid being obnoxious haven t had asthma symptoms since i q

In [1]:
1

1

In [3]:
import pandas as pd
tfs = pd.read_csv('/Users/eshwar/Downloads/predictions_with_subreddit_english.csv')
tfs.shape

(2831664, 643)